In [1]:
%pip install lxml
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
import gensim
from nltk.tokenize import word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

  Using cached lxml-6.0.0-cp312-cp312-win_amd64.whl.metadata (6.8 kB)
Using cached lxml-6.0.0-cp312-cp312-win_amd64.whl (4.0 MB)
Note: you may need to restart the kernel to use updated packages.


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\GOURAV\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\GOURAV\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df = pd.read_csv('Kindle_reviews/all_kindle_review.csv')
df.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [3]:
df = df[['reviewText', 'rating']].copy()
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [4]:
print(df.shape)

(12000, 2)


In [5]:
print(df.isnull().sum())

reviewText    0
rating        0
dtype: int64


In [6]:
df['rating'].unique()

array([3, 5, 4, 2, 1], dtype=int64)

In [7]:
df['rating'].value_counts()                  ### No imbalanced dataset 

rating
5    3000
4    3000
3    2000
2    2000
1    2000
Name: count, dtype: int64

In [8]:
## Preprocessing And Cleaning

In [9]:
## positive review 1 and negative review 0
df['rating'] = df['rating'].apply(lambda x:0 if x<3 else 1)

In [10]:
df['rating'].unique()

array([1, 0], dtype=int64)

In [11]:
df['rating'].value_counts()

rating
1    8000
0    4000
Name: count, dtype: int64

In [12]:
df[df['rating'] < 1]

,reviewText,rating
6,This has the makings of a good story... unfort...,0
10,"I received a copy of book to review. ""A secret...",0
12,I read the other reviews and decided to give i...,0
15,I was very disappointed with this book... not ...,0
17,How many times is author going to get Amazon t...,0
...,...,...
11982,"This book started off good, but somewhere alon...",0
11989,total crap. It was written as if the characte...,0
11990,I'll start off saying that I adore Jessica Par...,0
11991,"First of all, I've been to lots of different t...",0


In [13]:
## lowering all the cases

df['reviewText'] = df['reviewText'].str.lower()
df.head()

,reviewText,rating
0,"jace rankin may be short, but he's nothing to ...",1
1,great short read. i didn't want to put it dow...,1
2,i'll start by saying this is the first of four...,1
3,aggie is angela lansbury who carries pocketboo...,1
4,i did not expect this type of book to be in li...,1


In [14]:
stop_words = stopwords.words('english')

In [15]:
# removing special characters
df['reviewText'] = df['reviewText'].apply(lambda x:re.sub('[^a-z A-Z 0-9]', ' ', x))
# remove stop words
df['reviewText'] = df['reviewText'].apply(lambda x:' '.join([word for word in x.split() if word not in stop_words]))
# remove url
df['reviewText']=df['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(x)))
#remove html tags
df['reviewText'] = df['reviewText'].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text())
# remove additional spaces
df['reviewText']  = df['reviewText'].apply(lambda x: " ".join(x.split()))

In [16]:
df.head()

,reviewText,rating
0,jace rankin may short nothing mess man hauled ...,1
1,great short read want put read one sitting sex...,1
2,start saying first four books expecting 34 con...,1
3,aggie angela lansbury carries pocketbooks inst...,1
4,expect type book library pleased find price right,1


In [17]:
lematizer = WordNetLemmatizer()

In [18]:
def lemmatize_words(text):
    return " ".join([lematizer.lemmatize(word) for word in text.split()])

In [19]:
df['reviewText']=df['reviewText'].apply(lambda x:lemmatize_words(x))

In [20]:
df.head()

,reviewText,rating
0,jace rankin may short nothing mess man hauled ...,1
1,great short read want put read one sitting sex...,1
2,start saying first four book expecting 34 conc...,1
3,aggie angela lansbury carry pocketbook instead...,1
4,expect type book library pleased find price right,1


In [21]:
X_train, X_test, y_train, y_test = train_test_split(df['reviewText'], df['rating'], test_size=0.2, random_state=42)

In [22]:
X_train.shape

(9600,)

In [23]:
tokenize_reviews = [word_tokenize(word) for word in X_train]

In [24]:
tokenize_reviews

[['looking',
  'forward',
  'book',
  'came',
  'double',
  'space',
  'every',
  'paragraph',
  'kindle',
  'edition',
  'since',
  'action',
  'move',
  'around',
  'formatting',
  'make',
  'story',
  'hard',
  'follow',
  'die',
  'hard',
  'like',
  'want',
  'bother',
  'sad',
  'thing',
  'good',
  'book',
  'spoiled',
  'formatting',
  'fault',
  'author',
  'story',
  'good',
  'book',
  'energy',
  'read',
  'also',
  'emailed',
  'author'],
 ['already',
  'owned',
  'book',
  'spouse',
  'forgot',
  'already',
  'part',
  'library',
  'book',
  'unfortunate',
  'amazon',
  'safeguard',
  'place',
  'notify',
  'book',
  'already',
  'library'],
 ['cool',
  'forgot',
  'request',
  'rate',
  'came',
  'make',
  'mine',
  'unreliable',
  'rating',
  'doubt',
  'change',
  'rotation',
  'planet',
  'enough',
  'measure'],
 ['short',
  'short',
  'story',
  'basically',
  'scene',
  'party',
  'one',
  'night',
  'though',
  'admittedly',
  'entire',
  'book',
  'taken',
  'peri

In [25]:
model = gensim.models.Word2Vec(tokenize_reviews, vector_size=300,    
                                window=10,          
                                min_count=2,       
                                workers=4,
                                epochs=20,      
                                sg=1  )

In [26]:
model.wv.index_to_key

['book',
 'story',
 'read',
 'one',
 'character',
 'like',
 'good',
 'would',
 'really',
 'love',
 'time',
 'get',
 'author',
 'series',
 'reading',
 'well',
 'much',
 'first',
 'even',
 'short',
 'way',
 'could',
 'know',
 'sex',
 'great',
 'make',
 'little',
 'thing',
 'two',
 'want',
 'plot',
 'think',
 'romance',
 'also',
 'end',
 'find',
 'life',
 'go',
 'see',
 'enjoyed',
 'scene',
 'written',
 'never',
 'take',
 'kindle',
 'woman',
 'lot',
 'work',
 'thought',
 'many',
 'say',
 'year',
 'novel',
 'interesting',
 'found',
 'going',
 'bit',
 'liked',
 'loved',
 'give',
 'writing',
 'better',
 'review',
 'feel',
 'another',
 'man',
 'come',
 'hot',
 'star',
 'got',
 'though',
 'still',
 'enough',
 'back',
 'reader',
 'people',
 'made',
 'friend',
 'part',
 'something',
 'page',
 'bad',
 '34',
 'free',
 'keep',
 'world',
 'need',
 'new',
 'enjoy',
 'recommend',
 'relationship',
 'together',
 'next',
 'start',
 'felt',
 'best',
 'word',
 'put',
 'main',
 'looking',
 'however',
 'line

In [27]:
model.corpus_count

9600

In [28]:
model.wv.similar_by_word('novel')

[('book', 0.4009837806224823),
 ('liu', 0.387722909450531),
 ('lippman', 0.3816465139389038),
 ('assembled', 0.37923896312713623),
 ('redone', 0.3775946795940399),
 ('heroism', 0.37506353855133057),
 ('authour', 0.3745851218700409),
 ('story', 0.37261849641799927),
 ('simile', 0.3685212731361389),
 ('recapping', 0.3682071268558502)]

In [29]:
def avg_word2vec(doc):
    words = word_tokenize(doc)
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    return np.mean(word_vectors, axis=0) if word_vectors else np.zeros(model.wv.vector_size)

In [30]:
X_train_vectors = [avg_word2vec(doc) for doc in X_train]
X_test_vectors = [avg_word2vec(doc) for doc in X_test]

In [31]:
len(X_train_vectors), X_train_vectors[0].shape

(9600, (300,))

In [32]:
ml_model = LogisticRegression(max_iter=1000)
ml_model.fit(X_train_vectors, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [33]:
y_preds = ml_model.predict(X_test_vectors)

In [34]:
print("Classification report:", classification_report(y_test, y_preds))
print("Accuracy:", accuracy_score(y_test, y_preds))

Classification report:               precision    recall  f1-score   support

           0       0.81      0.66      0.72       803
           1       0.84      0.92      0.88      1597

    accuracy                           0.83      2400
   macro avg       0.83      0.79      0.80      2400
weighted avg       0.83      0.83      0.83      2400

Accuracy: 0.8329166666666666


In [35]:
## Bag of words

In [36]:
bow = CountVectorizer()
X_train_bow = bow.fit_transform(X_train)
X_test_bow = bow.transform(X_test)

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
X_train_tfidf=tfidf.fit_transform(X_train)
X_test_tfidf=tfidf.transform(X_test)

In [38]:
nb_model_bow=MultinomialNB().fit(X_train_bow,y_train)
nb_model_tfidf=MultinomialNB().fit(X_train_tfidf,y_train)

In [39]:
y_pred_bow=nb_model_bow.predict(X_test_bow)
y_pred_tfidf=nb_model_bow.predict(X_test_tfidf)

In [40]:
print("BOW accuracy: ",accuracy_score(y_test,y_pred_bow))
print("TFIDF accuracy: ",accuracy_score(y_test,y_pred_tfidf))

BOW accuracy:  0.8433333333333334
TFIDF accuracy:  0.8079166666666666
